In [2]:
# %load_ext autoreload
# %autoreload 2

In [3]:
import pandas as pd

In [4]:
from stellargraph import StellarGraph
from stellargraph.data import UnsupervisedSampler, EdgeSplitter, BiasedRandomWalk

[2022-02-05 14:07:09,687][tensorflow][DEBUG] Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.


2022-02-05 14:07:10.463398: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 14:07:10.471969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 14:07:10.472201: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-05 14:07:10.472911: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

# Load the data

In [5]:
from shared.schema import DatasetSchema, GraphSchema
from shared.graph.loading import pd_from_entity_schema


In [6]:
DATASET = DatasetSchema.load_schema('star-wars')
schema = GraphSchema.from_dataset(DATASET)

[2022-02-05 14:07:10,973][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetSchema'>, drop extra fields: True
[2022-02-05 14:07:10,973][simple_parsing.helpers.serialization.decoding][DEBUG] name = name, field_type = <class 'str'>
[2022-02-05 14:07:10,974][simple_parsing.helpers.serialization.decoding][DEBUG] name = database, field_type = <class 'str'>
[2022-02-05 14:07:10,975][simple_parsing.helpers.serialization.decoding][DEBUG] name = description, field_type = <class 'str'>
[2022-02-05 14:07:10,975][simple_parsing.helpers.serialization.decoding][DEBUG] name = versions, field_type = typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-05 14:07:10,976][simple_parsing.helpers.serialization.decoding][DEBUG] Decoding a Dict field: typing.Dict[str, shared.schema.dataset.DatasetVersion]
[2022-02-05 14:07:10,977][simple_parsing.helpers.serialization.serializable][DEBUG] from_dict for <class 'shared.schema.dataset.DatasetVe

In [7]:
explicit_label = False
explicit_timestamp = True
unix_timestamp = True
prefix_id = None
include_properties = lambda cs: [c for c in cs if c.startswith('feat_') or c == 'name']

nodes_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).set_index('id').drop(columns=['type']).sort_index()
    for label, entity_schema in schema.nodes.items()
}

edges_dfs = {
    label: pd_from_entity_schema(
        entity_schema,
        explicit_label=explicit_label,
        explicit_timestamp=explicit_timestamp,
        include_properties=include_properties,
        unix_timestamp=unix_timestamp,
        prefix_id=prefix_id,
    ).reset_index().drop(columns=['type']).drop_duplicates(subset=['src', 'dst', 'timestamp'])
    for label, entity_schema in schema.edges.items()
}

cursor = 0
for df in edges_dfs.values():
    df.index += cursor
    cursor += len(df)

In [8]:
graph = StellarGraph(
    nodes={k: df.drop(columns=['name_']) for k, df in nodes_dfs.items()},
    edges=edges_dfs,
    source_column='src',
    target_column='dst',
)
print(graph.info())

StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 2078

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [1120]
        Weights: all 1 (default)
        Features: float32 vector, length 2
    Character-INTERACTIONS->Character: [958]
        Weights: all 1 (default)
        Features: float32 vector, length 2


# Split dataset

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
edge_splitter_test = EdgeSplitter(graph)
graph_sub_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.05, method="global"
)

print(graph_sub_test.info())

** Sampled 103 positive and 103 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 1975

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [1062]
        Weights: all 1 (default)
        Features: none
    Character-INTERACTIONS->Character: [913]
        Weights: all 1 (default)
        Features: none


In [11]:
train_size = 0.75
val_size = 0.25

edge_splitter_train = EdgeSplitter(graph_sub_test)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)

(
    examples_train,
    examples_val,
    labels_train,
    labels_val,
) = train_test_split(examples, labels, train_size=train_size, test_size=val_size)

print(graph_train.info())

** Sampled 197 positive and 197 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 113, Edges: 1778

 Node types:
  Character: [113]
    Features: float32 vector, length 32
    Edge types: Character-INTERACTIONS->Character, Character-MENTIONS->Character

 Edge types:
    Character-MENTIONS->Character: [952]
        Weights: all 1 (default)
        Features: none
    Character-INTERACTIONS->Character: [826]
        Weights: all 1 (default)
        Features: none


In [12]:
pd.DataFrame(
    [
        (
            "Training Set",
            len(examples_train),
            "Train Graph",
            "Test Graph",
            "Train the Link Classifier",
        ),
        (
            "Validation Set",
            len(examples_val),
            "Train Graph",
            "Test Graph",
            "Validate the Link Classifier",
        ),
        (
            "Test set",
            len(examples_test),
            "Test Graph",
            "Full Graph",
            "Evaluate Link Classifier",
        ),
    ],
    columns=("Split", "Number of Examples", "Hidden from", "Picked from", "Use"),
).set_index("Split")


,Number of Examples,Hidden from,Picked from,Use
Split,,,,
Training Set,295,Train Graph,Test Graph,Train the Link Classifier
Validation Set,99,Train Graph,Test Graph,Validate the Link Classifier
Test set,206,Test Graph,Full Graph,Evaluate Link Classifier


# Train Attri2Vec

In [13]:
from tensorflow import keras

from stellargraph.mapper import Attri2VecLinkGenerator
from stellargraph.layer import Attri2Vec, link_classification

In [14]:
batch_size = 30
epochs = 100
walk_length = 5
walk_number = 2

In [15]:
unsupervised_samples = UnsupervisedSampler(
    graph_train, nodes=list(graph_train.nodes()), length=walk_length, number_of_walks=walk_number
)

generator = Attri2VecLinkGenerator(graph_train, batch_size)

In [16]:
layer_sizes = [128]

hinsage = Attri2Vec(
    layer_sizes=layer_sizes, generator=generator, bias=False, normalize=None
)

x_inp, x_out = hinsage.in_out_tensors()
prediction = link_classification(
    output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
)(x_out)

link_classification: using 'ip' method to combine node embeddings into edge embeddings


In [17]:
model = keras.Model(inputs=x_inp, outputs=prediction)
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss=keras.losses.binary_crossentropy,
    metrics=[keras.metrics.binary_accuracy],
)

In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 128)          4096        input_1[0][0]                    
__________________________________________________________________________________________________
output_embedding (Embedding)    (None, 1, 128)       14464       input_2[0][0]                    
______________________________________________________________________________________________

In [19]:
model.fit(
    generator.flow(unsupervised_samples),
    validation_data=generator.flow(examples_val, labels_val),
    epochs=epochs,
    verbose=2,
    use_multiprocessing=False,
    workers=4,
    shuffle=True,
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", patience=10, restore_best_weights=True
        )
    ],
)

Epoch 1/100


2022-02-05 14:07:11.866829: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-05 14:07:12.835777: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED
2022-02-05 14:07:12.835831: W tensorflow/core/framework/op_kernel.cc:1692] OP_REQUIRES failed at matmul_op_impl.h:438 : Internal: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node model/dense/MatMul (defined at tmp/ipykernel_33478/2964640790.py:1) ]] [Op:__inference_train_function_777]

Function call stack:
train_function


# Extract and cluster the embeddings

In [ ]:
embedding_model = keras.Model(inputs=x_inp, outputs=x_out)

In [ ]:
graph_node_list = graph.nodes()

node_embeddings = embedding_model.predict(
    generator.flow(list(zip(graph_node_list, graph_node_list)))
)
node_embeddings = node_embeddings[0]

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=0).fit(node_embeddings)

In [ ]:
from shared.constants import BENCHMARKS_RESULTS

save_path = BENCHMARKS_RESULTS.joinpath('analysis', 'node2vec-linkprediction-kmeans')
save_path.mkdir(parents=True, exist_ok=True)

In [ ]:
from shared.graph import CommunityAssignment

labeling = pd.Series(kmeans.labels_, index=graph_node_list, name="cid")
comlist = CommunityAssignment(labeling)

In [ ]:
comlist.save_comlist(save_path.joinpath('schema.comlist'))

In [ ]:
from datasets.scripts import export_to_visualization

export_to_visualization.run(
    export_to_visualization.Args(
        dataset='star-wars',
        version='base',
        run_paths=[str(save_path)]
    )
)

# Calculate Evaluation Metrics

In [ ]:
from shared.graph import DataGraph
from benchmarks.evaluation import get_metric_list

In [ ]:
G = DataGraph.from_schema(schema)

In [ ]:
metrics = get_metric_list(ground_truth=False, overlapping=False)

results = pd.DataFrame([
    {
        'metric': metric_cls.metric_name(),
        'value': metric_cls.calculate(G, comlist)
    }
    for metric_cls in metrics]
)
results